In [3]:
import pysftp
from urllib.parse import urlparse
import os

In [5]:
class Sftp:
    def __init__(self, hostname, username, password, port=22):
        self.connection = None
        self.hostname = hostname
        self.username = username
        self.password = password
        self.port = port
        self.cnopts = pysftp.CnOpts(knownhosts="C:/Users/JUAGUDELO/.ssh/known_hosts")
    def connect(self):
        """
        Connects to the sftp server and returns the sftp connection object
        """
        try:
            # Get the sftp connection object
            self.connection = pysftp.Connection(
                host=self.hostname,
                username=self.username,
                password=self.password,
                port=self.port,
                cnopts=self.cnopts,
            )
        except Exception as err:
            raise Exception(err)
        finally:
            print(f"Connected to {self.hostname} as {self.username}.")

    def disconnect(self):
        """Closes the sftp connection"""
        self.connection.close()
        print(f"Disconnected from host {self.hostname}")

    def listdir(self, remote_path):
        """lists all the files and directories in the specified path and returns them"""
        for obj in self.connection.listdir(remote_path):
            yield obj

    def listdir_attr(self, remote_path):
        """lists all the files and directories (with their attributes) in the specified path and returns them"""
        for attr in self.connection.listdir_attr(remote_path):
            yield attr

    def download(self, remote_path, target_local_path):
        """
        Downloads the file from remote sftp server to local.
        Also, by default extracts the file to the specified target_local_path
        """

        try:
            print(
                f"downloading from {self.hostname} as {self.username} [(remote path : {remote_path});(local path: {target_local_path})]"
            )

            # Create the target directory if it does not exist
            path, _ = os.path.split(target_local_path)
            if not os.path.isdir(path):
                try:
                    os.makedirs(path)
                except Exception as err:
                    raise Exception(err)

            # Download from remote sftp server to local
            self.connection.get(remote_path, target_local_path)
            print("download completed")

        except Exception as err:
            raise Exception(err)

In [6]:
import yaml
with open("../credentials.yaml", "r") as file:
    credentials = yaml.safe_load(file)

sftp = Sftp(hostname = credentials["hostname"],
            username = credentials["username"],
            password = credentials["password"],)

In [7]:
sftp.connect()
sftp.disconnect()


Connected to sftp-limagrain.northeurope.cloudapp.azure.com as SRV-SFTP-AGRIAL.


Exception: No hostkey for host sftp-limagrain.northeurope.cloudapp.azure.com found.